# Convert The Model from TendorFlow to Onnx

In [ ]:
#https://onnxruntime.ai/docs/tutorials/tf-get-started.html

In [ ]:
# Convert the model from tensorflow to onnx, --opset 13 is the best compatible version
!python -m tf2onnx.convert --saved-model saved_model --output objDetection.onnx --opset 13

0- 'detection_anchor_indices', 

1- 'detection_boxes', 

2- 'detection_classes',

3- 'detection_multiclass_scores', 

4- 'detection_scores', 

5- 'num_detections',  

6- 'raw_detection_boxes', 

7- 'raw_detection_scores'

# Check the model

In [ ]:
# https://stackoverflow.com/questions/75267445/why-does-onnxruntime-fail-to-create-cudaexecutionprovider-in-linuxubuntu-20/75267493#75267493
# https://github.com/microsoft/onnxruntime/issues/13264
import torch # This import is needed to run onnx on GPU 
from torch.utils.data import Dataset, DataLoader
import onnxruntime as rt
import onnx
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import cv2
import time

IMAGE_DIM = 300
CROPDIM = 50

In [ ]:
providers = ['CPUExecutionProvider'] #,'CUDAExecutionProvider'
model_name = 'objDetection.onnx'

In [ ]:
onnx_model = onnx.load(model_name)
onnx.checker.check_model(onnx_model)

In [ ]:
class GrayscaleImageDataset(Dataset):
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.image_files = os.listdir(folder_path)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_path = os.path.join(self.folder_path, self.image_files[index])
        return np.array(Image.open(image_path), dtype=np.uint8).reshape((1,300,300, 3))

In [ ]:
folder_path = '../datasets/dataset1'
dataset = GrayscaleImageDataset(folder_path)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True,)

In [ ]:
ort_sess = rt.InferenceSession(model_name,providers=providers)

In [ ]:
# optioan load a custom image
image_path="../datasets/dataset_random/img0.png"
input1 = np.array(Image.open(image_path)).reshape((1,300,300, 3))
plt.imshow(input1[0])


In [ ]:
# random number between 1 and dataset lenn

extraction = np.random.randint(0,dataset.__len__())
input1 = dataset.__getitem__(extraction)
results_ort = ort_sess.run(None, {"input_tensor": input1})
image = input1[0].copy()
crop = []



for index,vertex in enumerate(results_ort[1][0]): # i = [xmin, ymin, xmax, ymax]
    
    if(results_ort[4][0][index])<0.4: break; # skip if the confidence is too low
    vertex=vertex*IMAGE_DIM
    x_min,y_min,x_max,y_max = int(vertex[1]),int(vertex[0]),int(vertex[3]),int(vertex[2])
    
    start_point,end_point = (x_min,y_min),(x_max,y_max) # x,y

    center=(round((vertex[1]+vertex[3])/2),round((vertex[0]+vertex[2])/2))

    deltax = round(-((x_max-x_min)-CROPDIM)/2)
    deltay = round(-((y_max-y_min)-CROPDIM)/2)
    
    #check if the crop is out of the image
    errorx = 0
    errory = 0

    if (x_min - deltax <0 or x_max + deltax > IMAGE_DIM):  errorx = x_min - deltax if (x_min - deltax<0) else x_max + deltax-IMAGE_DIM
    if (y_min - deltay <0 or y_max + deltay > IMAGE_DIM):  errory = y_min - deltay if (y_min- deltay) else y_max + deltay -IMAGE_DIM

    
    new_start_point,new_end_point = (x_min-deltax -errorx,y_min-deltay-errory),(x_max+deltax- errorx,y_max+deltay-errory)
    
    image = cv2.rectangle(image, start_point, end_point, thickness=2, color=(0, 0, 255)) #old rect
    image = cv2.rectangle(image,new_start_point,new_end_point,thickness=1,color=(0,255,0)) #new rect
    image = cv2.circle(image, center, radius=1, color=(0, 0, 255), thickness=-1)
    crop.append(image[int(vertex[0])-deltay:int(vertex[2])+deltay, int(vertex[1])-deltax:int(vertex[3])+deltax]) #y,x

plt.imshow(image)

In [ ]:
for img in crop:
    resized = cv2.resize(img, (CROPDIM, CROPDIM), interpolation = cv2.INTER_AREA) 
    plt.imshow(resized)
    plt.show()